In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import pandas as pd
import os
import string
import random

In [16]:
df = pd.read_csv('stsds.csv')

In [17]:
df.head()

,sentence_A,sentence_B,relatedness_score
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,0.90
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,0.64
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0.94
3,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,0.68
4,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,0.74


In [9]:
#Hyperparams
lr = 1
gamma = 0.95
embed_size = 256
hidden_size = 512
num_epochs = 5

In [5]:
textcat = open("stsds-cat.txt").read()

In [6]:
vocab = Vocabulary(textcat)

In [7]:
vocab_size = vocab.size()

In [8]:
model = AttentionModel(embed_size, hidden_size, vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [ ]:
for e in range(num_epochs):
    total_loss = 0
    for i in range(len(df)):
        optimizer.zero_grad()
        
        t_a = torch.tensor(vocab.getSentenceArray(df["sentence_A"][i]))
        t_b = torch.tensor(vocab.getSentenceArray(df["sentence_B"][i]))
        
        out = model(t_a, t_b)
        loss = criterion(out, torch.tensor(df["relatedness_score"][i]).unsqueeze(0).unsqueeze(0))
        loss.backward()
        total_loss += loss.item()
        
        optimizer.step()
    print("Epoch", e, "Loss", total_loss)

/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 Loss 124.03386164454717
Epoch 1 Loss 106.82281311055762
Epoch 2 Loss 89.3954980608386
